<p align='right'><b>Ricardo Lebrón</b></p>

# Caso de estudio: *excessive number of floral organs*

En esta práctica utilizaremos el mapeo por secuenciación (*mapping-by-sequencing*) para encontrar la mutación responsable del fenotipo *excessive number of floral organs* (*eno*), el cual presenta alteraciones en el tamaño del meristemo floral que conducen al desarrollo de flores con órganos supernumerarios y a la formación de frutos multiloculares más grandes en tomate, como se muestra en la siguiente figura:

<br>

<center><img src="https://github.com/rlebron-bioinfo/ceiA3/raw/main/eno.png"/></center>

<br>

Por simplicidad, en esta práctica trataremos al rasgo *eno* como si de un rasgo monogénico se tratase, prescindiendo de la influencia de *LOCULE NUMBER* (*LC*). Adicionalmente, compararemos los niveles de expresión de plantas con fenotipo *eno* y con fenotipo *wild-type* para varios genes relacionados con el tamaño del fruto en tomate.

A lo largo de esta práctica se utilizará código en los lenguajes *Bash* y *Python* para analizar los conjuntos de datos de secuenciación y producir gráficos y tablas fácilmente interpretables. **No** es objetivo de esta práctica llegar a comprender dicho código, sino que se pretende que el alumno, una vez finalizada la práctica, sea capaz de:

- Identificar los diferentes tipos de ficheros que se utilizan para trabajar con datos de secuenciación y para qué se utilizan cada uno de ellos en el mapeo por secuenciación y el análisis de expresión.

- Comprender, a grandes rasgos, los pasos necesarios para identificar regiones genómicas candidatas que, probablemente, contengan las mutaciones responsables del fenotipo de interés. Para un rasgo monogénico, se esperan una única región genómica candidata y una o varias mutaciones que afecten a un único gen.

- Seleccionar posibles genes candidatos dentro de la región identificada, basándose en la presencia de mutaciones que, probablemente, afecten a su función y en el genotipo de plantas *eno* y plantas *wild-type* para dichas mutaciones.


# Preparación del entorno de trabajo

Antes de nada, vamos a preparar el entorno de trabajo que contiene todos los programas necesarios para llevar a cabo el mapeo por secuenciación y el análisis de expresión, así como el ensamblado genómico y los conjuntos de datos de secuenciación que vamos a utilizar. Esta preparación es completamente automática, sólo es necesario pulsar el botón de *play* (▶) de la siguiente celda y espera a que termine (tardará unos 5 minutos). Mientras tanto, presta atención a las indicaciones que te dará el profesor. Aprovecharemos los tiempos de espera para profundizar en los conceptos más importantes, resolver dudas y adelantar la explicación del siguiente paso.

In [ ]:
import os, sys, shutil, glob, warnings
warnings.filterwarnings('ignore')

def setup():
  if os.path.isdir('sample_data'):
    shutil.rmtree('sample_data')
  !wget https://repo.anaconda.com/miniconda/Miniconda3-py39_4.10.3-Linux-x86_64.sh &> log.txt
  !bash Miniconda3-py39_4.10.3-Linux-x86_64.sh -bfp /usr/local &>> log.txt
  print('✓ Miniconda instalada')
  sys.path.append('/usr/local/lib/python3.9/site-packages')
  !conda install -y -c bioconda hisat2 samtools bcftools bedtools star subread &>> log.txt
  !pip install statannotations &>> log.txt
  !pip install --upgrade --no-cache-dir gdown &>> log.txt
  print('✓ Dependencias instaladas')
  !gdown --q 1zEkHNzLgDT5dLCh5YC03-W4cz8COx2V7 && tar -xf ceiA3.tar.gz &>> log.txt
  print('✓ Ensamblado genómico y lecturas de secuenciación descargados')
  !rm -rf Miniconda3-py39_4.10.3-Linux-x86_64.sh ceiA3.tar.gz
  os.chdir(os.path.join(os.getcwd(), 'ceiA3'))
  !mkdir -p {D,R}NAseq/alignments \
  DNAseq/variant_calling DNAseq/mapping-by-sequencing \
  RNAseq/read_counts RNAseq/expression_analysis
  print('Entorno de trabajo preparado')

setup()

import matplotlib.pyplot as plt
from matplotlib import rc, rcParams
import matplotlib.ticker
from matplotlib.ticker import MaxNLocator
import seaborn as sns
import numpy as np
import pandas as pd
from csv import QUOTE_NONE
from itertools import combinations
from google.colab import drive
from statannotations.Annotator import Annotator

# Mapeo por secuenciación

La Genética Directa tiene por objetivo identificar las funciones que desempeñan los genes, partiendo del estudio de la herencia de fenotipos de interés y siguiendo un proceso que, habitualmente, se puede dividir en tres etapas:

1. Localizar el cromosoma o la región genómica que alberga las mutaciones que, probablemente, sean responsables del fenotipo de interés.

2. Identificar, dentro de dicha región genómica, las mutaciones responsables del fenotipo, basándose en su posible impacto sobre la función de uno o más genes.

3. Validar dicha identificación utilizando enfoques funcionales, como el silenciamiento mediante ARN de interferencia, la sobreexpresión génica o la edición del ADN mediante CRISPR/Cas9.

El mapeo por secuenciación ayuda en las dos primeras etapas y se basa en generar poblaciones de mapeo que segregan tanto individuos con el fenotipo de interés como individuos *wild-type*. El ADN de las plantas de la población de mapeo se agrupa en función del fenotipo y ambas agrupaciones se secuencian, por separado, mediante técnicas de secuenciación de segunda o de tercera generación. Una vez obtenidas las lecturas de secuenciación, se siguen los pasos indicados en el siguiente diagrama de flujo para detectar los genes candidatos:

<br>

<center><img src="https://github.com/rlebron-bioinfo/ceiA3/raw/main/workflow_ceiA3.png"/></center>

<br>

Vamos a describir este proceso para nuestro caso de estudio: el mutante *eno*. Al principio del proceso, disponemos de los siguientes ficheros dentro de la carpeta `ceiA3/DNAseq`:

- En la carpeta `assembly` están alojados los ficheros que contienen la secuencia del ensamblado genómico (para esta práctica, sólo utilizaremos el cromosoma 3 de tomate) (`SL4.0_ch03.fa`), sus índices (necesario para el alineamiento de las lecturas de secuenciación y la detección de variantes de secuencias) (`SL4.0_ch03.fa.fai` y ficheros con la extensión `.ht2`) y las anotaciones génicas que indican la localización genómica de los genes en el ensamblado (`ITAG4.0_gene_models.gff`). 

La secuencia del ensamblado genómico está en formato *FASTA*. Escribe **`head -c 100 DNAseq/assembly/SL4.0_ch03.fa`** en la siguiente celda para mostrar los cien primeros caracteres del fichero del ensamblado genómico. Consulta [este enlace](https://en.wikipedia.org/wiki/FASTA_format) para ver una descripción del formato *FASTA*. Mira el diagrama de flujo. **¿En qué etapas se necesita el ensamblado genómico? ¿Por qué crees que es necesario en estas etapas?**

Las anotaciones génicas están en formato *GFF3*. Escribe **`grep Solyc03g117230 DNAseq/assembly/ITAG4.0_gene_models.gff`** en la siguiente celda para mostrar las líneas del fichero de anotación correspondientes al gen *Solyc03g117230*. Consulta [este enlace](http://www.ensembl.org/info/website/upload/gff3.html) para ver una descripción del formato *GFF3*. **¿En qué etapas se necesitan las anotaciones génicas? ¿Por qué crees que son necesarias en estas etapas?**

- En la carpeta `reads`están alojadas las lecturas de secuenciación de la agrupación *eno* (`eno_1.fq.gz` y `eno_2.fq.gz`) y de la agrupación *wild-type* (`WT_1.fq.gz` y `WT_2.fq.gz`).

Las lecturas de secuenciación están formato *FASTQ* y hay dos ficheros por agrupación (terminados en `_1.fq.gz` y `_2.fq.gz`) porque se trata de lecturas *paired-end*, es decir, que cada lectura se compone de dos fragmentos secuenciados (cada uno en una hebra del ADN). Escribe este código en la siguiente celda para mostrar los dos fragmentos de la primera lectura de secuenciación de la agrupación *eno*:

**`zcat DNAseq/reads/eno_1.fq.gz | head -n 4`** <br>
**`zcat DNAseq/reads/eno_2.fq.gz | head -n 4`**

Consulta [este enlace](https://en.wikipedia.org/wiki/FASTQ_format) para ver una descripción del formato *FASTQ*. **¿En qué etapas se necesitan las lecturas de secuenciación? ¿Por qué crees que son necesarias en estas etapas?**

El resto de etapas se describen, sucesivamente, en los siguientes apartados.

In [ ]:
%%bash

# Escribe aquí tu código

## Alineamiento de lecturas de secuenciación

Antes de nada, ejecuta las dos siguientes celdas pulsando, en cada una de ellas, el botón de *play* (▶). Tardarán unos 5 minutos en total en ejecutarse.

En esta etapa, las lecturas de secuenciación *paired-end* en formato *FASTQ* se alinean con el ensamblado genómico de referencia (utilizando los índices `.ht2`) para descubrir:

1. La procedencia genómica de cada lectura, es decir, de qué localización del genoma proceden.

2. Las variantes de secuencia que portan con respecto al ensamblado genómico de referencia, es decir, las diferencias que hay entre la secuencia de la lectura y la secuencia de la localización correspondiente en el genoma de referencia.

En las dos siguientes celdas, se utiliza el programa [*HISAT2*](http://daehwankimlab.github.io/hisat2/) para alinear las lecturas de la agrupación *eno* y las lecturas de la agrupación *wild-type*, respectivamente, frente al genoma de referencia. Estos alineamientos están en formato [*SAM/BAM*](https://en.wikipedia.org/wiki/SAM_(file_format)) y se filtran, se ordenan y se indexan utilizan el programa [*SAMtools*](http://www.htslib.org/).

Una vez que terminen de ejecutarse las dos siguientes celdas, consulta su informe de alineamiento (aparecerá debajo de la propia celda). **¿Cuántas lecturas se han alineado correctamente (una sola vez) en el caso de la agrupación *eno*? ¿Y en la agrupación *wild-type*?**

In [ ]:
%%bash

BASENAME="DNAseq/reads/eno"
ASSEMBLY="DNAseq/assembly/SL4.0_ch03"
OUTPUT_DIR="DNAseq/alignments"
THREADS=2

RG=$(basename $BASENAME)

time hisat2 --rg SM:${RG} --rg-id $RG -x $ASSEMBLY \
-1 ${BASENAME}_1.fq.gz -2 ${BASENAME}_2.fq.gz -S ${OUTPUT_DIR}/${RG}.sam \
--threads $THREADS

samtools view -bhS -q 1 -@ $THREADS ${OUTPUT_DIR}/${RG}.sam | \
samtools sort -@ $THREADS -o ${OUTPUT_DIR}/${RG}.bam /dev/stdin && \
samtools index -@ $THREADS ${OUTPUT_DIR}/${RG}.bam && rm ${OUTPUT_DIR}/${RG}.sam


In [ ]:
%%bash

BASENAME="DNAseq/reads/WT"
ASSEMBLY="DNAseq/assembly/SL4.0_ch03"
OUTPUT_DIR="DNAseq/alignments"
THREADS=2

RG=$(basename $BASENAME)

time hisat2 --rg SM:${RG} --rg-id $RG -x $ASSEMBLY \
-1 ${BASENAME}_1.fq.gz -2 ${BASENAME}_2.fq.gz -S ${OUTPUT_DIR}/${RG}.sam \
--threads $THREADS

samtools view -bhS -q 1 -@ $THREADS ${OUTPUT_DIR}/${RG}.sam | \
samtools sort -@ $THREADS -o ${OUTPUT_DIR}/${RG}.bam /dev/stdin && \
samtools index -@ $THREADS ${OUTPUT_DIR}/${RG}.bam && rm ${OUTPUT_DIR}/${RG}.sam


## Detección de variantes de secuencia

Antes de nada, ejecuta la celda pulsando el botón de *play* (▶). Tardarán unos 25 minutos en ejecutarse. Si esta etapa tardara demasiado, puedes obtener el resultado a partir del punto de control correspondiente (ver la sección *Puntos de control* más abajo).

En esta etapa, se utilizan los alineamientos procedentes de la etapa anterior para detectar qué variantes de secuencia (con respecto al genoma de referencia) presentan las plantas de la agrupación *eno* y las plantas de la agrupación *wild-type*. Por tanto, para cada posición del genoma (en este caso, sólo del cromosoma 3) de tomate que presenten variantes en una u otra agrupación, dispondremos de la siguiente información:

- **Cromosoma y posición en el genoma de referencia en los que se encuentra la variante de secuencia.** Recuerda que en esta práctica sólo se estudia el cromosoma 3 de tomate.

- **Alelo en el genoma de referencia y alelos alternativos.** Es posible que haya más de un alelo alternativo (separados por coma en el fichero *VCF*). Filtraremos las variantes para quedarnos sólo con aquellas que sean bialélicas en la agrupación *eno*, en la agrupación *wild-type* o en ambas.

- **Genotipo en la agrupación de plantas *eno*.** En el formato *VCF*, un punto (*.*) indica que no se dispone de información para esta posición en esta muestra o agrupación, mientras que el número *0* indica que se trata del alelo de referencia y un número distinto de 0 indica que se trata de un alelo alternativo (normalmente, *1*). Así, *0/0* indica que se trata de plantas homocigotas para el alelo de referencia, *1/1* indica que se trata de plantas homocigotas para el primer alelo alternativo y *0/1* indica que son heterocigotas para los dos alelos antes mencionados. En esta práctica, asumiremos que el fenotipo de interés tiene una herencia homogénica recesiva. Por tanto, estamos interesados en variantes con genotipo *1/1* en la agrupación de plantas *eno*.  

- **Genotipo en la agrupación de plantas *wild-type* (*WT*).** En esta práctica, asumiremos que el fenotipo de interés tiene una herencia homogénica recesiva. Por tanto, estamos interesados en variantes con genotipo *0/0* ó *0/1* en la agrupación de plantas *wild-type*.

En la siguiente celda, se utiliza el programa [*BCFtools*](http://www.htslib.org/) para detectar las variantes de secuencia de la agrupación *eno* y de la agrupación *wild-type*, respectivamente, frente al genoma de referencia, utilizando las lecturas alineadas de una u otra agrupación. Estas variantes se reportan en el formato [*VCF*](https://en.wikipedia.org/wiki/Variant_Call_Format).


In [ ]:
%%bash

ALIGNMENTS="DNAseq/alignments/eno.bam DNAseq/alignments/WT.bam"
ASSEMBLY="DNAseq/assembly/SL4.0_ch03.fa"
VARIANTS="DNAseq/variant_calling/eno.vcf"
THREADS=2

time bcftools mpileup --threads $THREADS --redo-BAQ --min-BQ 30 --per-sample-mF \
--annotate FORMAT/AD,FORMAT/ADF,FORMAT/ADR,FORMAT/DP,FORMAT/SP,INFO/AD,INFO/ADF,INFO/ADR \
-f $ASSEMBLY $ALIGNMENTS \
| bcftools call --threads $THREADS --variants-only --multiallelic-caller --output-type v -o $VARIANTS

bcftools view --samples eno,WT --types snps \
--min-alleles 2 --max-alleles 2 --exclude 'GT[*]=="mis" | FMT/DP[*]<1' -Ou $VARIANTS \
| bcftools query --print-header --format '%CHROM\t%POS\t[%AD{0}\t%AD{1}\t]\n' \
-o ${VARIANTS%.vcf}.tsv


## Detección de regiones genómicas candidatas

A continuación, llevaremos a cabo el mapeo por secuenciación propiamente dicho. Para ello, calcularemos la frecuencia alélica para la agrupación *eno* y para la agrupación *wild-type* de todas las posiciones bialélicas en, al menos, una de las dos agrupaciones. A continuación, se calcula la frecuencia alélica media utilizando una ventana móvil no solapante de 1 Mpb para ambas agrupaciones por separado y se representan gráficamente.

La mayor parte del genoma debería tener una frecuencia alélica media en torno al 0.5 (50%) en ambas agrupaciones. En cambio, en las regiones candidatas la frecuencia alélica media de la agrupación *eno* se aproxima a 0, mientras que la frecuencia alélica media de la agrupación *wild-type* se mantiene en torno a 0.5.

Ejecuta la siguiente celda. **¿Se observa alguna región candidata? ¿Dónde está?**

Comprueba si tu propuesta de región candidata es adecuada cambiando a `True` el valor de la variable `where_is_ENO`.  

In [ ]:
where_is_ENO = False

variants = "DNAseq/variant_calling/eno.tsv"
png_plot = "DNAseq/mapping-by-sequencing/eno_ceiA3_ch03.png"
pdf_plot = "DNAseq/mapping-by-sequencing/eno_ceiA3_ch03.pdf"

step = 10**6
chrom = 3

df = pd.read_csv(variants, sep='\t')
df.columns = ['chrom', 'pos', 'mut_REF', 'mut_ALT', 'wt_REF', 'wt_ALT', '_']
del df['_']
df['mut_ALT'] = df['mut_ALT']/(df['mut_REF']+df['mut_ALT'])
df['wt_ALT'] = df['wt_ALT']/(df['wt_REF']+df['wt_ALT'])
del df['mut_REF'], df['wt_REF']
df['delta'] = df['mut_ALT']-df['wt_ALT']

font = {'size' : 10}
rc('font', **font)

fig, ax = plt.subplots(figsize=(7, 6))
fig.patch.set_facecolor('white')
plt.locator_params(axis='y', nbins=3)

df_by_chrom = dict()
for i in range(chrom, chrom+1):
    i = str(i).zfill(2)
    df_by_chrom[f'ch{i}'] = df[df['chrom'] == f'SL4.0ch{i}']

ch = str(chrom).zfill(2)
ch = f'ch{ch}'
bins = list(range(0,np.max(df_by_chrom[ch]['pos'])+10**6, step))
df_by_chrom[ch]['region'] = pd.cut(df_by_chrom[ch]['pos'], bins)
data_wt = df_by_chrom[ch].groupby('region')['wt_ALT'].mean().dropna()
data_wt = pd.DataFrame(data_wt)
data_wt['position'] = [(i+1)*0.5+(0.5*i) for i in range(len(data_wt))]
data_mut = df_by_chrom[ch].groupby('region')['mut_ALT'].mean().dropna()
data_mut = pd.DataFrame(data_mut)
data_mut['position'] = [(i+1)*0.5+(0.5*i) for i in range(len(data_mut))]
ax.plot(data_wt['position'], data_wt['wt_ALT'], 'royalblue', linewidth=3, label='WT')
ax.plot(data_mut['position'], data_mut['mut_ALT'], 'crimson', linewidth=3, label='$\it{eno}$')
ax.set_ylim([-0.0001,1.0001])
ax.set_xlim([-0.0001,None])

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.axhline(y=0.5, color='#AFAFAF', linewidth=2, linestyle='dotted')

if where_is_ENO:
  plt.axvline(60.8, ymax=0.90, color='black', linewidth=2, linestyle='solid')
  ax.text(60.8, 0.93, '$\mathit{ENO}\ ( \mathit{Solyc03g117230})$', ha='center', va='center', rotation='horizontal', color='black', weight='bold')

ax.set_ylabel('Average allele frequency', labelpad=10)
ax.set_xlabel(f'Chromosome {chrom} (Mb)', labelpad=10)
leg = ax.legend(bbox_to_anchor=(0,0.9,0.5,0), loc="lower center", ncol=2, handletextpad=0.5, columnspacing=0.75,
                facecolor='white', framealpha=0)
leg.get_frame().set_linewidth(0.0)
plt.xticks(rotation=0)
plt.yticks(rotation=0)

plt.tight_layout()
plt.savefig(png_plot, dpi=600, bbox_inches='tight')
plt.savefig(pdf_plot, dpi=600, bbox_inches='tight')

## Detección de genes candidatos

Una vez detectada la región candidata, vamos a filtrar las variantes de secuencia dentro de dicha región para tratar de encontrar genes candidatos, probablemente afectados por variantes de secuencia que sólo están en homocigosis en la agrupación *eno*. Para ello, llevaremos a cabo un filtrado en tres pasos consecutivos:

1. Seleccionaremos aquellas variantes de secuencia que estén ubicadas dentro de la región candidata. Por defecto, en la siguiente celda se indica que dicha región va desde los 50 Mpb hasta los 70 Mpb del cromosoma 3 de tomate. Modifica esta región si lo estimas oportuno y ejecuta las dos siguientes celdas. **¿Cuántas variantes de secuencia hay en la región candidata?**

In [ ]:
variants = "DNAseq/variant_calling/eno.vcf"
gene_annotations = "DNAseq/assembly/ITAG4.0_gene_models.gff"

# REGIÓN GENÓMICA CANDIDATA
chrom = "SL4.0ch03"
start = 50_000_000
end   = 70_000_000

!bedtools intersect -wa -wb -a $variants -b $gene_annotations | cut -f 1,2,4,5,10,11,14,15,16,18,20 > df
df = pd.read_csv('df', sep='\t', header=None)

samples = !grep -m 1 '#CHROM' $variants
samples = [f'{sample}_genotype' for sample in samples[0].strip().split('\t')[-2:]]
header = ['chrom', 'variant_pos', 'ref_allele', 'alt_allele'] + samples + ['feature', 'feat_start', 'feat_end', 'feat_strand', 'feat_ID']
df.columns = header

for sample in samples:
  df[sample] = df.apply(lambda row: row[sample].split(':')[0], axis=1)
df['feat_ID'] = df.apply(lambda row: row['feat_ID'].split('ID=')[-1].split(';')[0], axis=1)
!rm df


In [ ]:
# SELECCIÓN DE VARIANTES EN REGIÓN CANDIDATAS
df_region = df[(df['chrom'] == chrom) & (df['variant_pos'] >= start) & (df['variant_pos'] <= end)]
n_region = df_region.shape[0]
tsv = "DNAseq/mapping-by-sequencing/1_selected_variants_-_region.tsv"
df_region.to_csv(tsv, sep='\t', header=True, index=False, quoting=QUOTE_NONE)
print(f'Variantes seleccionadas: {df_region.shape[0]:,}')
df_region

2. De entre las variantes seleccionadas en el paso *1*, seleccionaremos aquellas variantes que tengan genotipo *1/1* en la agrupación *eno* y genotipo *0/0* ó *0/1* en la agrupación *wild-type*. Ejecuta la siguiente celda y observa el resultado. **¿Cuántas variantes de secuencia cumplen con estos requisitos?**

In [ ]:
# SELECCIÓN DE VARIANTES CON GENOTIPO 1/1 EN eno Y 0/0 ó 0/1 EN WT
df_genotype = df_region[(df_region['eno_genotype'] == '1/1') & (df_region['WT_genotype'].isin(['0/0', '0/1']))]
n_genotype = df_genotype.shape[0]
tsv = "DNAseq/mapping-by-sequencing/2_selected_variants_-_genotype.tsv"
df_genotype.to_csv(tsv, sep='\t', header=True, index=False, quoting=QUOTE_NONE)
print(f'Variantes seleccionadas: {df_genotype.shape[0]:,}')
df_genotype

3. De entre las variantes seleccionadas en el paso *2*, seleccionaremos aquellas variantes que estén ubicadas dentro de regiones codificantes (*CDS*). Ejecuta la siguiente celda y observa el resultado. **¿Cuántas variantes de secuencia están ubicadas dentro de regiones codificantes?**

Para poder obtener esta información, previamente se ha comprobado qué variantes solapan con las anotaciones génicas, utilizando el programa [BEDtools](https://bedtools.readthedocs.io/en/latest/). Este código está disponible en la primera celda de la sección de *Detección de genes candidatos*.

In [ ]:
# SELECCIÓN DE VARIANTES EN REGIONES CODIFICANTES
df_cds = df_genotype[df_genotype['feature'] == 'CDS']
n_cds = df_cds.shape[0]
tsv = "DNAseq/mapping-by-sequencing/3_selected_variants_-_CDS.tsv"
df_cds.to_csv(tsv, sep='\t', header=True, index=False, quoting=QUOTE_NONE)
print(f'Variantes seleccionadas: {df_cds.shape[0]:,}')
df_cds

Por último, vamos a comprobar cuáles de estas variantes de secuencia afectan al gen *Solyc03g117230*, un factor de transcripción de la superfamilia AP2/ERF previamente indentificado como el gen responsable del fenotipo *eno*. Ejecuta las dos siguientes celdas para realizar esta consulta y obtener un resumen de los resultados.

In [ ]:
# SELECCIÓN DE VARIANTES EN Solyc03g117230
df_ENO = df_cds[df_cds['feat_ID'].str.contains('Solyc03g117230')]
n_ENO = df_ENO.shape[0]
tsv = "DNAseq/mapping-by-sequencing/4_selected_variants_-_ENO.tsv"
df_ENO.to_csv(tsv, sep='\t', header=True, index=False, quoting=QUOTE_NONE)
print(f'Variantes seleccionadas: {df_ENO.shape[0]:,}')
df_ENO

In [ ]:
print(f'Número total de variantes en la región candidata: {n_region:,}')
print(f' - Con genotipo 1/1 en eno y 0/0 ó 0/1 en WT: {n_genotype:,}')
print(f'   - En regiones codificantes: {n_cds:,}')
print(f'     - En Solyc03g117230: {n_ENO:,}')

# Análisis de expresión génica (opcional)

En las siguientes celdas, se utilizan conjuntos de lecturas de secuenciación del transcriptoma de tres plantas con fenotipo *eno* y de tres plantas con fenotipo *wild-type* para medir y comparar los niveles de expresión de varios genes relacionados con el tamaño del fruto en tomate. Ejecuta las tres siguientes celdas (tardará unos 10 minutos) y trata de contestar a las preguntas.

In [ ]:
%%bash

INPUT_DIR="RNAseq/reads"
ASSEMBLY="RNAseq/assembly/"
GENE_ANNOTATIONS="RNAseq/assembly/ITAG4.0_9Genes.gtf"
OUTPUT_DIR="RNAseq/alignments"
COUNTS="RNAseq/read_counts/eno.tsv"
THREADS=2

for F in $INPUT_DIR/*_1.fq.gz
do
  F=$(basename $F)
  R=${F/_1.fq.gz/_2.fq.gz}
  STAR --runThreadN $THREADS --genomeDir $ASSEMBLY \
  --sjdbGTFfile $GENE_ANNOTATIONS --sjdbOverhang 149 \
  --readFilesIn $INPUT_DIR/$F $INPUT_DIR/$R \
  --readFilesCommand zcat --quantMode TranscriptomeSAM GeneCounts &> /dev/null \
  && samtools view -bhS Aligned.out.sam > ${F%_1.fq.gz}.bam \
  && samtools sort -@ $THREADS ${F%_1.fq.gz}.bam > ${F%_1.fq.gz}.sorted.bam \
  && samtools index -@ $THREADS ${F%_1.fq.gz}.sorted.bam \
  && mv ${F%_1.fq.gz}.sorted.bam ${OUTPUT_DIR}/${F%_1.fq.gz}.bam \
  && mv ${F%_1.fq.gz}.sorted.bam.bai ${OUTPUT_DIR}/${F%_1.fq.gz}.bam.bai \
  && rm -r _STARgenome *.sam *.bam *.out *.tab
done

featureCounts -T 64 -p -t exon -g gene_id -a $GENE_ANNOTATIONS \
-o $COUNTS `ls $OUTPUT_DIR/*.bam` &> /dev/null


In [ ]:
df = pd.read_csv('RNAseq/read_counts/eno.tsv', sep='\t', comment='#')
df.columns = [os.path.basename(os.path.splitext(col)[0]) for col in df.columns]
df['Geneid'] = df.apply(lambda row: row['Geneid'].split(':')[-1].split('.')[0], axis=1)  
df['Length'] = df['Length']/1_000
for col in df.filter(regex=('^eno|WT')).columns:
    df[col] /= df['Length']
    sf = np.sum(df[col])/1_000_000
    df[col] /= sf
df.set_index('Geneid', inplace=True)
df.drop(columns=['Chr','Start','End','Strand','Length'], inplace=True)
df = df[(df.T != 0).any()]
df

In [ ]:
png_plot = "RNAseq/expression_analysis/eno_ceiA3.png"
pdf_plot = "RNAseq/expression_analysis/eno_ceiA3.pdf"

def format_data(df):
  data = {
    'Genotype' : list(),
    'Feature'  : list(),
    'Value'    : list()
  }
  for feature in df.iloc[:, 2:].columns:
    for genotype in df.iloc[:, 0].unique():
      tmp = df[df.iloc[:, 0] == genotype][feature].to_numpy()
      tmp = tmp[~np.isnan(tmp)]
      n = tmp.shape[0]
      if n > 1:
        data['Genotype'].extend([genotype] * n)
        data['Feature'].extend([feature] * n)
        data['Value'].extend(list(tmp))
      elif n == 1:
        data['Genotype'].extend([genotype] * 2)
        data['Feature'].extend([feature] * 2)
        data['Value'].extend(list(tmp) * 2)  
      else:
        data['Genotype'].extend([genotype] * 2)
        data['Feature'].extend([feature] * 2)
        data['Value'].extend([0] * 2)        
  return pd.DataFrame(data)

plt.rcParams["figure.figsize"] = (16, 8)
plt.rcParams["figure.facecolor"] = 'white'

plot_df = df.T.reset_index()
plot_df.columns = ['sample'] + list(plot_df.columns)[1:]
plot_df['genotype'] = plot_df.apply(lambda row: row['sample'].split('_')[0], axis=1)
plot_df = plot_df[['genotype'] + [col for col in plot_df.columns if col != 'genotype']]
data = format_data(plot_df)
n = data['Genotype'].unique().shape[0]
order = data['Feature'].unique()
hue_order = data['Genotype'].unique()

sns.stripplot(x='Feature', y='Value', hue='Genotype', data=data, edgecolor='black', linewidth=2, jitter=False, dodge=True,
              size=5, marker='o', palette=['white']*n, alpha=1)
ax = sns.barplot(x='Feature', y='Value', hue='Genotype', data=data,
                 order=order, hue_order=hue_order, errcolor='black', palette=['crimson', 'royalblue'])
sns.despine()
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[2:4], labels[2:4], frameon=False)
ax.set_xlabel('')

pairs = list()
for feature in order:
  for genotype1, genotype2 in combinations(hue_order, 2):
    pairs.append(((feature, genotype1), (feature, genotype2)))

annot = Annotator(ax, pairs, data=data, x='Feature', y='Value', hue='Genotype', order=order, hue_order=hue_order)
annot.configure(test='t-test_ind', text_format='star', loc='inside', verbose=2)
annot.apply_test()
annot.annotate()

annot.configure(test='t-test_ind', text_format='full', loc='inside', verbose=2)
annot.apply_test()

labels = [item.get_text() for item in ax.get_xticklabels()]
pvalues = annot.get_annotations_text()
for i, p in zip(range(len(labels)), pvalues):
  labels[i] = labels[i] + '\n\n' + p.replace('t-test_ind ', '')
ax.set_xticklabels(labels)

plt.savefig(png_plot, dpi=600, bbox_inches='tight')
plt.savefig(pdf_plot, dpi=600, bbox_inches='tight')

- ¿Se expresan todos los genes estudiados en plantas con uno u otro fenotipo?
- ¿Cuál es el gen que más se expresa en plantas con fenotipo *wild-type*? ¿Y en plantas con fenotipo *eno*?
- ¿Qué genes están sobreexpresados en plantas con fenotipo *eno* con respecto a las plantas con fenotipo *wild-type*?
- ¿Qué genes están infraexpresados en plantas con fenotipo *eno* con respecto a las plantas con fenotipo *wild-type*?

# Copia de seguridad

En los apartados dispones del código necesario para crear una copia de seguridad de tu progreso a tu unidad de Google Drive y restaurarla. Para que la copia de seguridad sea funcional es necesario preparar primero el entorno de trabajo.

## Crear copia de seguridad

In [ ]:
dirs = [
  'DNAseq/alignments',
  'DNAseq/variant_calling',
  'DNAseq/variant_calling',
  'DNAseq/mapping-by-sequencing',
  'RNAseq/alignments',
  'RNAseq/read_counts',
  'RNAseq/expression_analysis'
]

def create_backup():
  gdrive = '/content/drive'
  drive.mount(gdrive)
  gd_ceiA3 = os.path.join(gdrive, 'MyDrive', 'ceiA3')
  if not os.path.exists(gd_ceiA3):
    os.mkdir(gd_ceiA3)
  for d in dirs:
    nt_d = os.path.join(os.getcwd(), d)
    gd_d = os.path.join(gd_ceiA3, d)
    if os.listdir(nt_d):
      if os.path.exists(gd_d):
        if not os.listdir(gd_d):
          shutil.rmtree(gd_d)
          shutil.copytree(nt_d, gd_d)
        else:
          files = glob.glob(os.path.join(nt_d, '*'))
          for f in files:
            gfile = os.path.join(gd_d, os.path.basename(f))
            if not os.path.exists(gfile):
              shutil.copyfile(f, gfile)
      else:
        shutil.copytree(nt_d, gd_d)
  print('Copia de seguridad creada')

create_backup()

## Restaurar copia de seguridad

In [ ]:
dirs = [
  'DNAseq/alignments',
  'DNAseq/variant_calling',
  'DNAseq/variant_calling',
  'DNAseq/mapping-by-sequencing',
  'RNAseq/alignments',
  'RNAseq/read_counts',
  'RNAseq/expression_analysis'
]

def restore_backup():
  gdrive = '/content/drive'
  drive.mount(gdrive)
  gd_ceiA3 = os.path.join(gdrive, 'MyDrive', 'ceiA3')
  if os.path.exists(gd_ceiA3):
    for d in dirs:
      nt_d = os.path.join(os.getcwd(), d)
      gd_d = os.path.join(gd_ceiA3, d)
      if not os.listdir(nt_d):
        if os.path.exists(gd_d):
          shutil.rmtree(nt_d)
          shutil.copytree(gd_d, nt_d)
  print('Copia de seguridad restaurada')

restore_backup()

# Puntos de control

En los apartados dispones de puntos de control para las diferentes etapas del proceso. Si tu proceso tarda demasiado y no quieres esperar o tu sesión se ha cerrado y no habías guardado una copia de seguridad de tu progreso, puedes utilizar estos puntos de control para descargar el resultado de cada etapa. Para que los puntos de control sean funcionales es necesario preparar primero el entorno de trabajo (si no lo has hecho ya) y disponer de los resultados de las etapas anteriores.

## Alineamiento de lecturas de secuenciación

In [ ]:
# DNAseq/alignments

files = [
  '1-33vRzpKB6aR9FidTvPWttwru6nrjiaN',
  '1-8VhcssuOznghrNiRdCh1ZyAbKJntCar',
  '1-GCAXn2-H-cnqdnS-KIDSWguu4IynMyg',
  '1-VqT3WfvdYeaPl-4Z-JvR_sIvgKl42eH'
]

for ID in files:
  !cd DNAseq/alignments && gdown --q $ID &>> log.txt
print('Punto de control restaurado')

## Detección de variantes de secuencia

In [ ]:
# DNAseq/variant_calling

files = [
  '100ipTPoykM3V1w6OwDKMZvGygDrTcoV2',
  '1-XDGIxFQXCk2PZI98Qyl7npKcOtxzlFW'
]

for ID in files:
  !cd DNAseq/variant_calling && gdown --q $ID &>> log.txt
print('Punto de control restaurado')

## Detección de regiones genómicas candidatas y de genes candidatos

In [ ]:
# DNAseq/mapping-by-sequencing

files = [
  '1-dXy-zKNCM5zCI-LP8bP9_IliRutN2Tr',
  '1-c3UMQaTowaBU-jzr9lNqGg3sZBkC3z8',
  '1-XyxQa0MMiQgrrRzC_urLfaOwXFepG-g',
  '1-ZFGjt0aiLEBAztuOdT2UBcpjLPk-P-P',
  '1-iyiOpRCpDgow9pCCqXz1rCUpjsEcC9k',
  '1-p1Prxkqn6wsUFXKPStZj6_MFz8hPqzp'
]

for ID in files:
  !cd DNAseq/mapping-by-sequencing && gdown --q $ID &>> log.txt
print('Punto de control restaurado')

## Análisis de expresión génica

In [ ]:
# RNAseq/alignments

files = [
  '10avbMBlkriF10nipTAuEq6sbpKqiDR13',
  '10ad0oE_OqysW4K5R9vyz9E_ci0nSOnIZ',
  '10USK_-yFvGhofoYjSEmAW2ZQCDFtM8qP',
  '10YvlQHP5IDcgQZNpswiS7Kbt5Oeqb9J1',
  '10TDjGhE4U-QUgbQdvCfppV-Oyfdpg0pB',
  '10QpHZQiS96sf67CpGEzdpWAakv9YfWxc',
  '10KytY5biOf-PHmd45Kzt1pbfflDKpbLu',
  '10MEh2v829wIsiAQPrQ3GSokFzpPYztms',
  '10HZu5-YrFbeGuoi4by8Cp0FE39DnKOxg',
  '10EIkpgqn1qwykxUpwN8zoEmiPhwC6jyF',
  '109BiuGysKD0goPXrzfJN-XaQB4gPpRRr',
  '105yGW3A9kRJD6SNGeY25LP4x20Woekws'
]

for ID in files:
  !cd RNAseq/alignments && gdown --q $ID &>> log.txt
print('Punto de control restaurado')

In [ ]:
# RNAseq/read_counts

files = [
  '102UFRLgPkUmL4qPdTvNx9AUGbExDtvws',
  '102yCjFRUjqen-1CES5gb6gzukV9IINu5'
]

for ID in files:
  !cd RNAseq/read_counts && gdown --q $ID &>> log.txt
print('Punto de control restaurado')

In [ ]:
# RNAseq/expression_analysis

files = [
  '101LmMY7p34g5dCvPO-QrMrMqe5DDQwev',
  '101mSauMHXMGVuDPVsiKQP2aF8TMgGgwU'
]

for ID in files:
  !cd RNAseq/expression_analysis && gdown --q $ID &>> log.txt
print('Punto de control restaurado')

# Bibliografía

- Yuste-Lisbona, Fernando J., et al. "Effective Mapping by Sequencing to Isolate Causal Mutations in the Tomato Genome." Crop Breeding. Humana, New York, NY, 2021. 89-103.

- Yuste-Lisbona, Fernando J., et al. "ENO regulates tomato fruit size through the floral meristem development network." Proceedings of the National Academy of Sciences 117.14 (2020): 8187-8195.